In [9]:
import nltk
from nltk.corpus import brown
from nltk import pos_tag
def pos_features(sentence, i, history): 
    features = {"suffix(1)": sentence[i][-1:],"suffix(2)": sentence[i][-2:],"suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features 
class ConsecutivePosTagger(nltk.TaggerI): 
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:untagged_sent = nltk.tag.untag(tagged_sent)
        history = []
        for i, (word, tag) in enumerate(tagged_sent):
            featureset = pos_features(untagged_sent, i, history)
            train_set.append( (featureset, tag) )
            history.append(tag)
            self.classifier = nltk.NaiveBayesClassifier.train(train_set)
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history) 
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history) 
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size] 
tagger = ConsecutivePosTagger(train_sents) 
print(tagger.evaluate(test_sents))

/tmp/ipykernel_34484/2713575892.py:34: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(tagger.evaluate(test_sents))


0.27714418173388966


In [2]:
import nltk
def punct_features(tokens, i):
     return {'next-word-capitalized': tokens[i+1][0].isupper(), ...             'prevword': tokens[i-1].lower(), ...             'punct': tokens[i], ...             'prev-word-is-one-char': len(tokens[i-1]) == 1} 

In [10]:
import nltk
from nltk.grammar import CFG, Production
from nltk import word_tokenize, pos_tag
def generate_cfg(sentence):
    # Tokenize the sentence and perform part-of-speech tagging
    tokens = word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)  # Fix: Use nltk.pos_tag instead of pos_tag

    # Initialize lists to store grammar productions
    productions = []

    # Add productions for each tagged token
    for i in range(len(tagged_tokens)):
        token, pos_tag = tagged_tokens[i]
        # For the first token, add a production for the start symbol S
        if i == 0:
            productions.append(Production(nltk.Nonterminal('S'), [nltk.Nonterminal(pos_tag)]))
        # Add a production for each subsequent token
        productions.append(Production(nltk.Nonterminal(pos_tag), [token]))

    # Create a CFG from the list of productions
    cfg = CFG(nltk.Nonterminal('S'), productions)
    return cfg

# Example sentences
sentences = [
    "The cat sat on the mat.",
    "Birds sing melodiously in the morning.",
    "Children play happily in the park."
]

# Generate CFGs for each sentence and print them
for sentence in sentences:
    cfg = generate_cfg(sentence)
    print("CFG for sentence:", sentence)
    print(cfg)
    print()

CFG for sentence: The cat sat on the mat.
Grammar with 8 productions (start state = S)
    S -> DT
    DT -> 'The'
    NN -> 'cat'
    VBD -> 'sat'
    IN -> 'on'
    DT -> 'the'
    NN -> 'mat'
    . -> '.'

CFG for sentence: Birds sing melodiously in the morning.
Grammar with 8 productions (start state = S)
    S -> NNS
    NNS -> 'Birds'
    VBG -> 'sing'
    RB -> 'melodiously'
    IN -> 'in'
    DT -> 'the'
    NN -> 'morning'
    . -> '.'

CFG for sentence: Children play happily in the park.
Grammar with 8 productions (start state = S)
    S -> NNP
    NNP -> 'Children'
    VBP -> 'play'
    RB -> 'happily'
    IN -> 'in'
    DT -> 'the'
    NN -> 'park'
    . -> '.'

